In [1]:
# 1-1. 데이터 가져오기 

import pandas as pd 
import matplotlib.pyplot as plt 


socar = pd.read_csv('../../0. raw_data/insurance_fraud_detect_data.csv')
socar 

,fraud_YN,car_model,sharing_type,age_group,has_previous_accident,cumulative_use_count,b2b,accident_ratio,pf_type,socarpass,...,insure_cost,accident_location,car_part1,car_part2,repair_cnt,acc_type1,insurance_site_aid_YN,police_site_aid_YN,total_prsn_cnt,test_set
0,0,2,1,3,0,2,0,100,2,0,...,0,1,0,0,1,0,0,0,-1,0
1,0,1,0,4,0,4,0,0,3,1,...,0,1,1,1,2,1,1,1,0,1
2,0,1,0,4,1,4,0,100,3,1,...,12925040,3,1,0,2,1,1,1,0,1
3,0,3,1,3,0,4,0,0,2,0,...,0,3,1,0,1,0,0,0,-1,0
4,0,1,0,1,0,1,0,100,1,0,...,1209410,1,0,0,2,0,0,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,0,2,0,2,0,2,0,100,2,0,...,0,1,1,0,1,0,0,0,-1,0
15996,0,2,0,2,1,4,0,100,1,1,...,0,2,1,0,1,0,0,0,-1,0
15997,0,2,1,2,0,4,0,100,1,0,...,0,0,0,0,1,0,0,0,-1,1
15998,0,2,0,2,0,2,0,100,2,0,...,0,4,1,0,1,0,0,0,-1,0


In [2]:
# 테스트 셋 분리

df = socar[socar['test_set']==0]
df_test = socar[socar['test_set']==1]

In [3]:
# test_set 분리 
X_test = df_test.drop('fraud_YN', axis = 1)
y_test = df_test['fraud_YN']

In [4]:
# train-valid 분리 

from sklearn.model_selection import train_test_split

y_target = df['fraud_YN']
X_features = df.drop('fraud_YN', axis = 1)

X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size = 0.2, random_state = 13, stratify = y_target)

# 비율 확인 
print(y_train.value_counts()/y_train.shape[0]*100) 
print(y_valid.value_counts()/y_valid.shape[0]*100)

0    99.73794
1     0.26206
Name: fraud_YN, dtype: float64
0    99.728261
1     0.271739
Name: fraud_YN, dtype: float64


In [5]:
# 학습 / 예측 / 평가 함수 

# 평가 함수 

from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score

def get_clf_eval(y_test, pred):
    # confusion = confusion_matrix(y_test, pred) #오차행렬
    accuracy = accuracy_score(y_test,pred) # 정확도 
    precision = precision_score(y_test,pred) # 정밀도
    recall = recall_score(y_test,pred) # 재현율 
    f1 = f1_score(y_test,pred)
    roc_auc = roc_auc_score(y_test,pred)
    return accuracy, precision, recall, f1, roc_auc

# 성능 DF 반환 

def get_result(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    pred = model.predict(X_test)

    return get_clf_eval(y_test, pred)

# 다수의 모델 성능 비교 

def get_result_df(models, model_names, X_train, y_train, X_test, y_test):
    col_names = ['Accuracy', 'precision', 'recall', 'f1', 'roc']
    tmp = []
    for model in models:
        tmp.append(get_result(model, X_train, y_train, X_test, y_test))

    return pd.DataFrame(tmp, columns=col_names, index=model_names)


# 성능 확인 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import LinearSVC


# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()


# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 


,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.50000
DecisionTree,0.997283,0.0,0.0,0.0,0.50000
RandomFore,0.997283,0.0,0.0,0.0,0.50000
LGBM,0.997283,0.0,0.0,0.0,0.50000
SVC,0.981366,0.0,0.0,0.0,0.49202


## Trial #1
- ssscale / pca(n=2) / smote-teen

In [6]:
# 모델 객체를 predict(X_test) 시켜줄 것 
df_1 = df.copy()

In [7]:
y_target = df_1['fraud_YN']
X_features = df_1.drop('fraud_YN', axis=1)

In [8]:
y_target = df_1['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [9]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=2)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.tail()


,component_1,component_2,target
12874,-0.900400,-0.052815,0
12875,-1.223356,1.352673,0
12876,-0.676312,0.381217,0
12877,-1.383713,0.656432,0
12878,-1.286983,-1.030856,0


In [10]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 


,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.996894,0.0,0.0,0.0,0.499805
RandomFore,0.997283,0.0,0.0,0.0,0.500000
LGBM,0.996118,0.0,0.0,0.0,0.499416
SVC,0.997283,0.0,0.0,0.0,0.500000


In [11]:
# 복합 샘플링 적용 

from imblearn.combine import SMOTEENN
sme = SMOTEENN(random_state=13)
X_train_sme, y_train_sme = sme.fit_sample(X_train, y_train)


model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train_sme, y_train_sme, X_valid, y_valid)

results

,Accuracy,precision,recall,f1,roc
LogiReg,0.650233,0.005531,0.714286,0.010977,0.682172
DecisionTree,0.477873,0.001490,0.285714,0.002965,0.382055
RandomFore,0.782997,0.001805,0.142857,0.003565,0.463799
LGBM,0.788820,0.003697,0.285714,0.007299,0.537953
SVC,0.664596,0.005767,0.714286,0.011442,0.689373


In [12]:
# 테스트셋 적용 
# PCA / ss 

df_test_scaled = scaler.transform(X_test)
df_test_pca = pca.transform(df_test_scaled)

pca_columns = ['component_1','component_2']

DF_test_pca = pd.DataFrame(df_test_pca, columns=pca_columns)
DF_test_pca['target'] = y_target 
DF_test_pca.tail()

X_test_scaled = DF_test_pca.drop('target', axis=1)
y_test_scaled = DF_test_pca['target']

In [13]:
# 다수의 모델 테스트 성능 비교 함수 

def get_test_result(model, X_test_scaled, y_test_scaled):
    pred = model.predict(X_test_scaled)
    return get_clf_eval(y_test_scaled, pred)

def get_test_df(models, model_names, X_test_scaled, y_test_scaled):
    col_names = ['Accuracy', 'precision', 'recall', 'f1', 'roc']
    tmp = []
    for model in models:
        tmp.append(get_test_result(model, X_test_scaled,y_test_scaled))
    return pd.DataFrame(tmp, columns=col_names, index=model_names)


In [14]:
# 테스트 셋 모델 적용 

models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_test_df(models,model_names, X_test_scaled, y_test_scaled)
results 

,Accuracy,precision,recall,f1,roc
LogiReg,0.453701,0.011641,0.740741,0.022923,0.595968
DecisionTree,0.441205,0.006893,0.444444,0.013575,0.442810
RandomFore,0.742711,0.006361,0.185185,0.012300,0.466381
LGBM,0.755207,0.006693,0.185185,0.012920,0.472683
SVC,0.467158,0.011350,0.703704,0.022340,0.584399


## Trial #2
## ssscale / pca(n=2) / smote-tomek

In [15]:
df_2 = df.copy()

y_target = df_2['fraud_YN']
X_features = df_2.drop('fraud_YN', axis = 1)

y_target = df_2['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [16]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=2)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.tail()


,component_1,component_2,target
12874,-0.900405,-0.060917,0
12875,-1.223357,1.348270,0
12876,-0.676330,0.384071,0
12877,-1.383707,0.658953,0
12878,-1.286992,-1.024010,0


In [17]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)

results 


,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.996506,0.0,0.0,0.0,0.499611
RandomFore,0.996894,0.0,0.0,0.0,0.499805
LGBM,0.994953,0.0,0.0,0.0,0.498832
SVC,0.997283,0.0,0.0,0.0,0.500000


In [18]:
# 복합 샘플링 적용 

from imblearn.combine import SMOTETomek
tmk = SMOTETomek(random_state=13)
X_train_tmk, y_train_tmk = tmk.fit_sample(X_train, y_train)


model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train_tmk, y_train_tmk, X_valid, y_valid)

results

,Accuracy,precision,recall,f1,roc
LogiReg,0.687500,0.006188,0.714286,0.012270,0.700856
DecisionTree,0.478261,0.001491,0.285714,0.002967,0.382250
RandomFore,0.825311,0.004474,0.285714,0.008811,0.556248
LGBM,0.833851,0.007026,0.428571,0.013825,0.631763
SVC,0.698758,0.006418,0.714286,0.012723,0.706501


In [19]:
# 테스트셋 적용 
# PCA / ss 

df_test_scaled = scaler.transform(X_test)
df_test_pca = pca.transform(df_test_scaled)

pca_columns = ['component_1','component_2']

DF_test_pca = pd.DataFrame(df_test_pca, columns=pca_columns)
DF_test_pca['target'] = y_target 
DF_test_pca.tail()

X_test_scaled = DF_test_pca.drop('target', axis=1)
y_test_scaled = DF_test_pca['target']

In [20]:
# 테스트 셋 모델 적용 

models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_test_df(models,model_names, X_test_scaled, y_test_scaled)
results 

,Accuracy,precision,recall,f1,roc
LogiReg,0.493111,0.010699,0.629630,0.021040,0.560775
DecisionTree,0.446331,0.006384,0.407407,0.012571,0.427039
RandomFore,0.799423,0.006590,0.148148,0.012618,0.476627
LGBM,0.807434,0.008562,0.185185,0.016367,0.499024
SVC,0.497597,0.011414,0.666667,0.022444,0.581394


## Trial #3
## ssscale / pca(n=3) / smoteenn

In [21]:
df_3 = df.copy()

y_target = df_3['fraud_YN']
X_features = df_3.drop('fraud_YN', axis=1)

y_target = df_3['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [22]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=3)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2','component_3']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.tail()

,component_1,component_2,component_3,target
12874,-0.900491,-0.045188,1.060802,0
12875,-1.223137,1.336833,0.032581,0
12876,-0.676498,0.361288,1.549271,0
12877,-1.382150,0.629489,-0.848013,0
12878,-1.287963,-1.011051,-1.559397,0


In [23]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 


,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.995342,0.0,0.0,0.0,0.499027
RandomFore,0.997283,0.0,0.0,0.0,0.500000
LGBM,0.997283,0.0,0.0,0.0,0.500000
SVC,0.997283,0.0,0.0,0.0,0.500000


In [24]:
# 복합 샘플링 적용 

from imblearn.combine import SMOTEENN
sme = SMOTEENN(random_state=13)
X_train_sme, y_train_sme = sme.fit_sample(X_train, y_train)


model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train_sme, y_train_sme, X_valid, y_valid)

results

,Accuracy,precision,recall,f1,roc
LogiReg,0.618012,0.005066,0.714286,0.010060,0.666018
DecisionTree,0.494953,0.002308,0.428571,0.004591,0.461853
RandomFore,0.911491,0.008889,0.285714,0.017241,0.599455
LGBM,0.910714,0.008811,0.285714,0.017094,0.599066
SVC,0.630823,0.005241,0.714286,0.010406,0.672441


In [25]:
# 테스트셋 적용 
# PCA / ss 

df_test_scaled = scaler.transform(X_test)
df_test_pca = pca.transform(df_test_scaled)

pca_columns = ['component_1','component_2','component_3']

DF_test_pca = pd.DataFrame(df_test_pca, columns=pca_columns)
DF_test_pca['target'] = y_target 
DF_test_pca.tail()

X_test_scaled = DF_test_pca.drop('target', axis=1)
y_test_scaled = DF_test_pca['target']

In [26]:
# 테스트 셋 모델 적용 

models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_test_df(models,model_names, X_test_scaled, y_test_scaled)
results 

,Accuracy,precision,recall,f1,roc
LogiReg,0.495674,0.009512,0.555556,0.018703,0.525354
DecisionTree,0.522909,0.005413,0.296296,0.010631,0.410592
RandomFore,0.900673,0.006969,0.074074,0.012739,0.490980
LGBM,0.899071,0.003448,0.037037,0.006309,0.471815
SVC,0.497917,0.008929,0.518519,0.017555,0.508128


## Trial #4
## ssscale / pca(n=3) / smote-tomek

In [27]:
df_4 = df.copy()

y_target = df_4['fraud_YN']
X_features = df_4.drop('fraud_YN', axis=1)

y_target = df_4['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [28]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=3)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2','component_3']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.tail()

,component_1,component_2,component_3,target
12874,-0.900546,-0.054369,1.033132,0
12875,-1.223094,1.348247,0.040208,0
12876,-0.676095,0.382813,1.497658,0
12877,-1.383486,0.656945,-0.858279,0
12878,-1.287146,-1.025008,-1.597878,0


In [29]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 


,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.996894,0.0,0.0,0.0,0.499805
RandomFore,0.997283,0.0,0.0,0.0,0.500000
LGBM,0.997283,0.0,0.0,0.0,0.500000
SVC,0.997283,0.0,0.0,0.0,0.500000


In [30]:
# 복합 샘플링 적용 

from imblearn.combine import SMOTETomek
tmk = SMOTETomek(random_state=13)
X_train_tmk, y_train_tmk = tmk.fit_sample(X_train, y_train)


model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train_tmk, y_train_tmk, X_valid, y_valid)

results

,Accuracy,precision,recall,f1,roc
LogiReg,0.677407,0.005995,0.714286,0.011891,0.695796
DecisionTree,0.484860,0.003012,0.571429,0.005993,0.528026
RandomFore,0.938276,0.006494,0.142857,0.012422,0.541650
LGBM,0.942158,0.006944,0.142857,0.013245,0.543597
SVC,0.683618,0.006112,0.714286,0.012121,0.698910


In [31]:
# 테스트셋 적용 
# PCA / ss 

df_test_scaled = scaler.transform(X_test)
df_test_pca = pca.transform(df_test_scaled)

pca_columns = ['component_1','component_2', 'component_3']

DF_test_pca = pd.DataFrame(df_test_pca, columns=pca_columns)
DF_test_pca['target'] = y_target 
DF_test_pca.tail()

X_test_scaled = DF_test_pca.drop('target', axis=1)
y_test_scaled = DF_test_pca['target']

In [32]:
# 테스트 셋 모델 적용 

models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_test_df(models,model_names, X_test_scaled, y_test_scaled)
results 

,Accuracy,precision,recall,f1,roc
LogiReg,0.553989,0.009346,0.481481,0.018336,0.518052
DecisionTree,0.508811,0.005256,0.296296,0.010329,0.403481
RandomFore,0.935918,0.005714,0.037037,0.009901,0.490400
LGBM,0.935598,0.000000,0.000000,0.000000,0.471881
SVC,0.555912,0.010094,0.518519,0.019802,0.537378


## Trial #5
## robust / pca(n=2) / smoteenn

In [33]:
df_5 = df.copy()

y_target = df_5['fraud_YN']
X_features = df_5.drop('fraud_YN', axis=1)

y_target = df_5['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [34]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=2)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.head()


,component_1,component_2,target
0,-10.878061,-1.681139,0
1,89.088264,-0.446654,0
2,-10.919831,0.595497,0
3,-10.861503,0.626787,1
4,89.146163,-0.591036,0


In [35]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 

,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.996894,0.0,0.0,0.0,0.499805
RandomFore,0.997283,0.0,0.0,0.0,0.500000
LGBM,0.996506,0.0,0.0,0.0,0.499611
SVC,0.997283,0.0,0.0,0.0,0.500000


In [36]:
# 복합 샘플링 적용 

from imblearn.combine import SMOTEENN
sme = SMOTEENN(random_state=13)
X_train_sme, y_train_sme = sme.fit_sample(X_train, y_train)


model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train_sme, y_train_sme, X_valid, y_valid)

results

,Accuracy,precision,recall,f1,roc
LogiReg,0.139363,0.002700,0.857143,0.005384,0.497275
DecisionTree,0.814053,0.006276,0.428571,0.012371,0.621837
RandomFore,0.772516,0.005128,0.428571,0.010135,0.601012
LGBM,0.763199,0.003295,0.285714,0.006515,0.525107
SVC,0.139363,0.002700,0.857143,0.005384,0.497275


In [37]:
# 테스트셋 적용 
# PCA / robust 

df_test_scaled = scaler.transform(X_test)
df_test_pca = pca.transform(df_test_scaled)

pca_columns = ['component_1','component_2']

DF_test_pca = pd.DataFrame(df_test_pca, columns=pca_columns)
DF_test_pca['target'] = y_target 
DF_test_pca.tail()

X_test_scaled = DF_test_pca.drop('target', axis=1)
y_test_scaled = DF_test_pca['target']

In [38]:
# 테스트 셋 모델 적용 

models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_test_df(models,model_names, X_test_scaled, y_test_scaled)
results 

,Accuracy,precision,recall,f1,roc
LogiReg,0.144185,0.008550,0.851852,0.016930,0.494930
DecisionTree,0.796540,0.015924,0.370370,0.030534,0.585314
RandomFore,0.753284,0.010540,0.296296,0.020356,0.526784
LGBM,0.745594,0.010217,0.296296,0.019753,0.522906
SVC,0.144185,0.008550,0.851852,0.016930,0.494930


## Trial #6
## robust / pca(n=3) / smoteenn

In [39]:
df_6 = df.copy()

y_target = df_6['fraud_YN']
X_features = df_6.drop('fraud_YN', axis=1)

y_target = df_6['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [40]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=3)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2','component_3']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.head()


,component_1,component_2,component_3,target
0,-10.878061,-1.681139,-0.444496,0
1,89.088264,-0.446654,3.490084,0
2,-10.919831,0.595497,0.765326,0
3,-10.861503,0.626787,-1.255630,1
4,89.146163,-0.591036,-0.507473,0


In [41]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 

,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.996118,0.0,0.0,0.0,0.499416
RandomFore,0.997283,0.0,0.0,0.0,0.500000
LGBM,0.996894,0.0,0.0,0.0,0.499805
SVC,0.997283,0.0,0.0,0.0,0.500000


In [42]:
# 복합 샘플링 적용 

from imblearn.combine import SMOTEENN
sme = SMOTEENN(random_state=13)
X_train_sme, y_train_sme = sme.fit_sample(X_train, y_train)


model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train_sme, y_train_sme, X_valid, y_valid)

results

,Accuracy,precision,recall,f1,roc
LogiReg,0.143634,0.002714,0.857143,0.005410,0.499416
DecisionTree,0.736025,0.004418,0.428571,0.008746,0.582717
RandomFore,0.891693,0.007246,0.285714,0.014134,0.589529
LGBM,0.888199,0.007018,0.285714,0.013699,0.587777
SVC,0.145186,0.002719,0.857143,0.005420,0.500195


In [43]:
# 테스트셋 적용 
# PCA / robust 

df_test_scaled = scaler.transform(X_test)
df_test_pca = pca.transform(df_test_scaled)

pca_columns = ['component_1','component_2','component_3']

DF_test_pca = pd.DataFrame(df_test_pca, columns=pca_columns)
DF_test_pca['target'] = y_target 
DF_test_pca.tail()

X_test_scaled = DF_test_pca.drop('target', axis=1)
y_test_scaled = DF_test_pca['target']

In [44]:
# 테스트 셋 모델 적용 

models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_test_df(models,model_names, X_test_scaled, y_test_scaled)
results 

,Accuracy,precision,recall,f1,roc
LogiReg,0.148350,0.008592,0.851852,0.017012,0.497031
DecisionTree,0.689202,0.011399,0.407407,0.022177,0.549534
RandomFore,0.861583,0.012048,0.185185,0.022624,0.526335
LGBM,0.848766,0.015251,0.259259,0.028807,0.556585
SVC,0.149311,0.008601,0.851852,0.017031,0.497516


## Trial #7
## robust / pca(n=5) / smoteenn

In [45]:
df_7 = df.copy()

y_target = df_7['fraud_YN']
X_features = df_7.drop('fraud_YN', axis=1)

y_target = df_7['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [46]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=5)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2','component_3','component_4','component_5']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.head()


,component_1,component_2,component_3,component_4,component_5,target
0,-10.878061,-1.681139,-0.444494,-0.810866,0.028940,0
1,89.088264,-0.446654,3.490084,-1.120898,-0.444883,0
2,-10.919831,0.595497,0.765326,0.844330,-1.570778,0
3,-10.861503,0.626787,-1.255626,1.676016,2.764579,1
4,89.146163,-0.591036,-0.507475,0.475744,2.254303,0


In [47]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 

,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.995730,0.0,0.0,0.0,0.499221
RandomFore,0.997283,0.0,0.0,0.0,0.500000
LGBM,0.997283,0.0,0.0,0.0,0.500000
SVC,0.997283,0.0,0.0,0.0,0.500000


In [48]:
# 복합 샘플링 적용 

from imblearn.combine import SMOTEENN
sme = SMOTEENN(random_state=13)
X_train_sme, y_train_sme = sme.fit_sample(X_train, y_train)


model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train_sme, y_train_sme, X_valid, y_valid)

results

,Accuracy,precision,recall,f1,roc
LogiReg,0.665373,0.004635,0.571429,0.009195,0.618529
DecisionTree,0.551242,0.003457,0.571429,0.006873,0.561308
RandomFore,0.952252,0.008475,0.142857,0.016000,0.548657
LGBM,0.955745,0.000000,0.000000,0.000000,0.479175
SVC,0.657997,0.004535,0.571429,0.008999,0.614831


In [49]:
# 테스트셋 적용 
# PCA / robust 

df_test_scaled = scaler.transform(X_test)
df_test_pca = pca.transform(df_test_scaled)

pca_columns = ['component_1','component_2','component_3','component_4','component_5']

DF_test_pca = pd.DataFrame(df_test_pca, columns=pca_columns)
DF_test_pca['target'] = y_target 
DF_test_pca.tail()

X_test_scaled = DF_test_pca.drop('target', axis=1)
y_test_scaled = DF_test_pca['target']

In [50]:
# 테스트 셋 모델 적용 

models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_test_df(models,model_names, X_test_scaled, y_test_scaled)
results 

,Accuracy,precision,recall,f1,roc
LogiReg,0.561679,0.011653,0.592593,0.022857,0.577001
DecisionTree,0.449215,0.010417,0.666667,0.020513,0.556992
RandomFore,0.931432,0.010471,0.074074,0.018349,0.506494
LGBM,0.933355,0.010811,0.074074,0.018868,0.507464
SVC,0.559756,0.010893,0.555556,0.021368,0.557674


## Trial #8
## 불필요 feature 삭제 
## ssscale / pca(n=2) / smoteenn

In [51]:
df_8 = df.copy()
df_8 = df_8.drop(columns=['sharing_type', 'has_previous_accident','b2b','accident_ratio','socarpass', 'car_part2'])
df_8

y_target = df_8['fraud_YN']
X_features = df_8.drop('fraud_YN', axis=1)

y_target = df_8['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [52]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=2)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.tail()


,component_1,component_2,target
12874,-0.997271,-0.389117,0
12875,-1.084393,1.182813,0
12876,-1.041032,-0.183276,0
12877,-1.248365,0.124023,0
12878,-1.167924,-1.638932,0


In [53]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 


,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.996894,0.0,0.0,0.0,0.499805
RandomFore,0.997283,0.0,0.0,0.0,0.500000
LGBM,0.996506,0.0,0.0,0.0,0.499611
SVC,0.997283,0.0,0.0,0.0,0.500000


In [54]:
# 복합 샘플링 적용 

from imblearn.combine import SMOTEENN
sme = SMOTEENN(random_state=13)
X_train_sme, y_train_sme = sme.fit_sample(X_train, y_train)


model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train_sme, y_train_sme, X_valid, y_valid)

results

,Accuracy,precision,recall,f1,roc
LogiReg,0.725932,0.005658,0.571429,0.011204,0.648891
DecisionTree,0.629270,0.002101,0.285714,0.004171,0.457960
RandomFore,0.824146,0.004444,0.285714,0.008753,0.555664
LGBM,0.829581,0.004587,0.285714,0.009029,0.558388
SVC,0.731366,0.005772,0.571429,0.011429,0.651615


## Trial #9
## 불필요 feature 삭제
## ssscale / pca(n=2) / smotetomek

In [55]:
df_9 = df.copy()
df_9 = df_9.drop(columns=['sharing_type', 'has_previous_accident','b2b','accident_ratio','socarpass', 'car_part2'])
df_9

y_target = df_9['fraud_YN']
X_features = df_9.drop('fraud_YN', axis=1)

y_target = df_9['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [56]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=2)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.tail()


,component_1,component_2,target
12874,-0.997455,-0.429202,0
12875,-1.084396,1.179995,0
12876,-1.041136,-0.196668,0
12877,-1.248378,0.121996,0
12878,-1.167814,-1.607929,0


In [57]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)

results 


,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.996894,0.0,0.0,0.0,0.499805
RandomFore,0.997283,0.0,0.0,0.0,0.500000
LGBM,0.996894,0.0,0.0,0.0,0.499805
SVC,0.997283,0.0,0.0,0.0,0.500000


In [58]:
# 복합 샘플링 적용 

from imblearn.combine import SMOTETomek
tmk = SMOTETomek(random_state=13)
X_train_tmk, y_train_tmk = tmk.fit_sample(X_train, y_train)


model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train_tmk, y_train_tmk, X_valid, y_valid)

results

,Accuracy,precision,recall,f1,roc
LogiReg,0.732531,0.005797,0.571429,0.011478,0.652199
DecisionTree,0.717003,0.002755,0.285714,0.005457,0.501946
RandomFore,0.863354,0.002882,0.142857,0.005650,0.504087
LGBM,0.860637,0.002825,0.142857,0.005540,0.502725
SVC,0.736413,0.005882,0.571429,0.011645,0.654146


## Trial #10
## 불필요 feature 삭제
## ssscale / pca(n=3) / smoteenn

In [59]:
df_10 = df.copy()
df_10 = df_10.drop(columns=['sharing_type', 'has_previous_accident','b2b','accident_ratio','socarpass', 'car_part2'])
df_10

y_target = df_10['fraud_YN']
X_features = df_10.drop('fraud_YN', axis=1)

y_target = df_10['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [60]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=3)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2','component_3']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.tail()

,component_1,component_2,component_3,target
12874,-0.997338,-0.391127,-0.122761,0
12875,-1.084459,1.186246,-1.040574,0
12876,-1.041145,-0.188972,1.261117,0
12877,-1.248418,0.112574,0.418119,0
12878,-1.167909,-1.644492,1.514023,0


In [61]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 


,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.5
DecisionTree,0.997283,0.0,0.0,0.0,0.5
RandomFore,0.997283,0.0,0.0,0.0,0.5
LGBM,0.997283,0.0,0.0,0.0,0.5
SVC,0.997283,0.0,0.0,0.0,0.5


In [62]:
# 복합 샘플링 적용 

from imblearn.combine import SMOTEENN
sme = SMOTEENN(random_state=13)
X_train_sme, y_train_sme = sme.fit_sample(X_train, y_train)


model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train_sme, y_train_sme, X_valid, y_valid)

results

,Accuracy,precision,recall,f1,roc
LogiReg,0.692935,0.006297,0.714286,0.012484,0.703581
DecisionTree,0.394798,0.002564,0.571429,0.005105,0.482873
RandomFore,0.911879,0.000000,0.000000,0.000000,0.457182
LGBM,0.909161,0.000000,0.000000,0.000000,0.455819
SVC,0.701475,0.006477,0.714286,0.012837,0.707863


## Trial #11
## 불필요 feature 삭제
## ssscale / pca(n=3) / smote-tomek

In [63]:
df_11 = df.copy()
df_11 = df_11.drop(columns=['sharing_type', 'has_previous_accident','b2b','accident_ratio','socarpass', 'car_part2'])
df_11

y_target = df_11['fraud_YN']
X_features = df_11.drop('fraud_YN', axis=1)

y_target = df_11['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [64]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=3)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2','component_3']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.tail()

,component_1,component_2,component_3,target
12874,-0.997232,-0.396969,-0.106826,0
12875,-1.084406,1.196095,-1.035607,0
12876,-1.041042,-0.192480,1.274568,0
12877,-1.248462,0.114674,0.411528,0
12878,-1.167995,-1.643799,1.503551,0


In [65]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 


,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.5
DecisionTree,0.997283,0.0,0.0,0.0,0.5
RandomFore,0.997283,0.0,0.0,0.0,0.5
LGBM,0.997283,0.0,0.0,0.0,0.5
SVC,0.997283,0.0,0.0,0.0,0.5


In [66]:
# 복합 샘플링 적용 

from imblearn.combine import SMOTETomek
tmk = SMOTETomek(random_state=13)
X_train_tmk, y_train_tmk = tmk.fit_sample(X_train, y_train)


model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train_tmk, y_train_tmk, X_valid, y_valid)

results

,Accuracy,precision,recall,f1,roc
LogiReg,0.726708,0.005674,0.571429,0.011236,0.649280
DecisionTree,0.399845,0.002586,0.571429,0.005148,0.485403
RandomFore,0.947981,0.000000,0.000000,0.000000,0.475282
LGBM,0.945652,0.000000,0.000000,0.000000,0.474114
SVC,0.728261,0.005706,0.571429,0.011299,0.650058


## Trial #12
## 불필요 feature 삭제
## ssscale / pca(n=4) / smoteen


In [76]:
df_12 = df.copy()
df_12 = df_12.drop(columns=['sharing_type', 'has_previous_accident','b2b','accident_ratio','socarpass', 'car_part2'])
df_12

y_target = df_12['fraud_YN']
X_features = df_12.drop('fraud_YN', axis=1)

y_target = df_12['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [77]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=4)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2','component_3','component_4']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.tail()

,component_1,component_2,component_3,component_4,target
12874,-0.997281,-0.389817,-0.116317,0.552664,0
12875,-1.084352,1.188931,-1.023974,-1.200307,0
12876,-1.041084,-0.187462,1.266980,0.511580,0
12877,-1.248455,0.111979,0.416285,-0.701233,0
12878,-1.167983,-1.646121,1.503822,0.213860,0


In [78]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 


,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.995730,0.0,0.0,0.0,0.499221
RandomFore,0.997283,0.0,0.0,0.0,0.500000
LGBM,0.996894,0.0,0.0,0.0,0.499805
SVC,0.997283,0.0,0.0,0.0,0.500000


In [79]:
# 복합 샘플링 적용 

from imblearn.combine import SMOTEENN
sme = SMOTEENN(random_state=13)
X_train_sme, y_train_sme = sme.fit_sample(X_train, y_train)


model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train_sme, y_train_sme, X_valid, y_valid)

results

,Accuracy,precision,recall,f1,roc
LogiReg,0.642469,0.005411,0.714286,0.010741,0.678279
DecisionTree,0.409161,0.002626,0.571429,0.005229,0.490074
RandomFore,0.940994,0.000000,0.000000,0.000000,0.471779
LGBM,0.936335,0.000000,0.000000,0.000000,0.469443
SVC,0.651786,0.005556,0.714286,0.011025,0.682951


## Trial #13
## 불필요 feature 삭제
## ssscale / pca(n=5) / smoteen

In [80]:
df_13 = df.copy()
df_13 = df_13.drop(columns=['sharing_type', 'has_previous_accident','b2b','accident_ratio','socarpass', 'car_part2'])
df_13

y_target = df_13['fraud_YN']
X_features = df_13.drop('fraud_YN', axis=1)

y_target = df_13['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [81]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=5)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2','component_3','component_4','component_5']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.tail()

,component_1,component_2,component_3,component_4,component_5,target
12874,-0.997281,-0.389814,-0.116329,0.552666,0.174954,0
12875,-1.084352,1.188922,-1.023928,-1.200317,-0.223161,0
12876,-1.041084,-0.187458,1.266952,0.511587,0.279710,0
12877,-1.248457,0.112034,0.416062,-0.701189,1.388940,0
12878,-1.167983,-1.646129,1.503840,0.213857,-1.619871,0


In [82]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 


,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.995730,0.0,0.0,0.0,0.499221
RandomFore,0.997283,0.0,0.0,0.0,0.500000
LGBM,0.997283,0.0,0.0,0.0,0.500000
SVC,0.997283,0.0,0.0,0.0,0.500000


In [84]:
# 복합 샘플링 적용 

from imblearn.combine import SMOTEENN
sme = SMOTEENN(random_state=13)
X_train_sme, y_train_sme = sme.fit_sample(X_train, y_train)


model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train_sme, y_train_sme, X_valid, y_valid)

results

,Accuracy,precision,recall,f1,roc
LogiReg,0.611025,0.004975,0.714286,0.009881,0.662515
DecisionTree,0.679348,0.002430,0.285714,0.004819,0.483067
RandomFore,0.967780,0.012821,0.142857,0.023529,0.556442
LGBM,0.958075,0.000000,0.000000,0.000000,0.480343
SVC,0.614907,0.005025,0.714286,0.009980,0.664461


## Trial #14
## 불필요 feature 삭제
## robustscale / pca(n=2) / smoteen

In [85]:
df_14 = df.copy()
df_14 = df_14.drop(columns=['sharing_type', 'has_previous_accident','b2b','accident_ratio','socarpass', 'car_part2'])
df_14

y_target = df_14['fraud_YN']
X_features = df_14.drop('fraud_YN', axis=1)

y_target = df_14['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [86]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=2)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.head()


,component_1,component_2,target
0,-1.572293,-0.336286,0
1,-1.330600,2.911762,0
2,0.710055,0.955343,0
3,0.723886,-1.467563,1
4,-1.492137,-1.360312,0


In [87]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 

,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.996506,0.0,0.0,0.0,0.499611
RandomFore,0.996894,0.0,0.0,0.0,0.499805
LGBM,0.996118,0.0,0.0,0.0,0.499416
SVC,0.997283,0.0,0.0,0.0,0.500000


In [88]:
# 복합 샘플링 적용 

from imblearn.combine import SMOTEENN
sme = SMOTEENN(random_state=13)
X_train_sme, y_train_sme = sme.fit_sample(X_train, y_train)


model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train_sme, y_train_sme, X_valid, y_valid)

results

,Accuracy,precision,recall,f1,roc
LogiReg,0.009317,0.002735,1.000000,0.005456,0.503309
DecisionTree,0.658773,0.003417,0.428571,0.006780,0.543986
RandomFore,0.809006,0.002053,0.142857,0.004049,0.476839
LGBM,0.798913,0.005803,0.428571,0.011450,0.614247
SVC,0.030280,0.002794,1.000000,0.005573,0.513819


## Trial #15
## 불필요 feature 삭제
## robustscale / pca(n=3) / smoteen

In [92]:
df_15 = df.copy()
df_15 = df_15.drop(columns=['sharing_type', 'has_previous_accident','b2b','accident_ratio','socarpass', 'car_part2'])
df_15

y_target = df_15['fraud_YN']
X_features = df_15.drop('fraud_YN', axis=1)

y_target = df_15['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [94]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=3)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2','component_3']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.head()


,component_1,component_2,component_3,target
0,-1.572293,-0.336294,0.612036,0
1,-1.330600,2.911791,0.048666,0
2,0.710055,0.955337,0.728995,0
3,0.723886,-1.467586,-3.026099,1
4,-1.492137,-1.360284,-2.742969,0


In [95]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 

,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.996506,0.0,0.0,0.0,0.499611
RandomFore,0.997283,0.0,0.0,0.0,0.500000
LGBM,0.996894,0.0,0.0,0.0,0.499805
SVC,0.997283,0.0,0.0,0.0,0.500000


In [96]:
# 복합 샘플링 적용 

from imblearn.combine import SMOTEENN
sme = SMOTEENN(random_state=13)
X_train_sme, y_train_sme = sme.fit_sample(X_train, y_train)


model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train_sme, y_train_sme, X_valid, y_valid)

results

,Accuracy,precision,recall,f1,roc
LogiReg,0.756599,0.006369,0.571429,0.012598,0.664266
DecisionTree,0.714286,0.004082,0.428571,0.008086,0.571818
RandomFore,0.896739,0.003831,0.142857,0.007463,0.520825
LGBM,0.888199,0.003534,0.142857,0.006897,0.516543
SVC,0.753494,0.006289,0.571429,0.012442,0.662709


## Trial #16
## 불필요 feature 삭제
## robustscale / pca(n=4) / smoteen

In [97]:
df_16 = df.copy()
df_16 = df_16.drop(columns=['sharing_type', 'has_previous_accident','b2b','accident_ratio','socarpass', 'car_part2'])
df_16

y_target = df_16['fraud_YN']
X_features = df_16.drop('fraud_YN', axis=1)

y_target = df_16['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [98]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=4)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2','component_3','component_4']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.head()


,component_1,component_2,component_3,component_4,target
0,-1.572293,-0.336297,0.612052,0.574432,0
1,-1.330600,2.911795,0.048638,0.958089,0
2,0.710055,0.955331,0.729032,-1.638588,0
3,0.723886,-1.467583,-3.026115,0.302512,1
4,-1.492137,-1.360288,-2.742943,1.302907,0


In [99]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 

,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.996894,0.0,0.0,0.0,0.499805
RandomFore,0.997283,0.0,0.0,0.0,0.500000
LGBM,0.996894,0.0,0.0,0.0,0.499805
SVC,0.997283,0.0,0.0,0.0,0.500000


In [100]:
# 복합 샘플링 적용 

from imblearn.combine import SMOTEENN
sme = SMOTEENN(random_state=13)
X_train_sme, y_train_sme = sme.fit_sample(X_train, y_train)


model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train_sme, y_train_sme, X_valid, y_valid)

results

,Accuracy,precision,recall,f1,roc
LogiReg,0.607143,0.004926,0.714286,0.009785,0.660568
DecisionTree,0.751941,0.006250,0.571429,0.012365,0.661931
RandomFore,0.948758,0.007874,0.142857,0.014925,0.546905
LGBM,0.943711,0.007143,0.142857,0.013605,0.544375
SVC,0.602873,0.004873,0.714286,0.009681,0.658427


## Trial #17
## 불필요 feature 삭제
## robustscale / pca(n=5) / smoteen

In [101]:
df_17 = df.copy()
df_17 = df_17.drop(columns=['sharing_type', 'has_previous_accident','b2b','accident_ratio','socarpass', 'car_part2'])
df_17

y_target = df_17['fraud_YN']
X_features = df_17.drop('fraud_YN', axis=1)

y_target = df_17['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [103]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=5)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2','component_3','component_4','component_5']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.head()


,component_1,component_2,component_3,component_4,component_5,target
0,-1.572293,-0.336297,0.612052,0.574432,-0.255082,0
1,-1.330600,2.911795,0.048638,0.958089,-1.164894,0
2,0.710055,0.955331,0.729032,-1.638588,0.411414,0
3,0.723886,-1.467583,-3.026115,0.302512,-0.146063,1
4,-1.492137,-1.360288,-2.742943,1.302907,-0.107516,0


In [104]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 

,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.996894,0.0,0.0,0.0,0.499805
RandomFore,0.997283,0.0,0.0,0.0,0.500000
LGBM,0.996894,0.0,0.0,0.0,0.499805
SVC,0.997283,0.0,0.0,0.0,0.500000


In [105]:
# 복합 샘플링 적용 

from imblearn.combine import SMOTEENN
sme = SMOTEENN(random_state=13)
X_train_sme, y_train_sme = sme.fit_sample(X_train, y_train)


model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train_sme, y_train_sme, X_valid, y_valid)

results

,Accuracy,precision,recall,f1,roc
LogiReg,0.635093,0.004251,0.571429,0.008439,0.603348
DecisionTree,0.551630,0.004318,0.714286,0.008584,0.632736
RandomFore,0.970885,0.014286,0.142857,0.025974,0.557999
LGBM,0.966615,0.012346,0.142857,0.022727,0.555858
SVC,0.627329,0.004162,0.571429,0.008264,0.599455


## Trial #18
## 불필요 feature 삭제
## ssscale / pca(n=2) / adasyn

In [106]:
df_18 = df.copy()
df_18 = df_18.drop(columns=['sharing_type', 'has_previous_accident','b2b','accident_ratio','socarpass', 'car_part2'])
df_18

y_target = df_18['fraud_YN']
X_features = df_18.drop('fraud_YN', axis=1)

y_target = df_18['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [107]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=2)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.tail()


,component_1,component_2,target
12874,-0.997303,-0.390287,0
12875,-1.084291,1.190410,0
12876,-1.041052,-0.188696,0
12877,-1.248556,0.110672,0
12878,-1.167965,-1.647353,0


In [108]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 

,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.997283,0.0,0.0,0.0,0.500000
RandomFore,0.997283,0.0,0.0,0.0,0.500000
LGBM,0.996506,0.0,0.0,0.0,0.499611
SVC,0.997283,0.0,0.0,0.0,0.500000


In [109]:
# 복합 샘플링 적용 

from imblearn.over_sampling import ADASYN

ads = ADASYN(random_state=13)
X_train_ads, y_train_ads = ads.fit_sample(X_train, y_train)


model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train_ads, y_train_ads, X_valid, y_valid)

results

,Accuracy,precision,recall,f1,roc
LogiReg,0.739907,0.005961,0.571429,0.011799,0.655897
DecisionTree,0.705357,0.002646,0.285714,0.005242,0.496107
RandomFore,0.862966,0.002874,0.142857,0.005634,0.503893
LGBM,0.866460,0.002950,0.142857,0.005780,0.505644
SVC,0.741071,0.005988,0.571429,0.011852,0.656481


## Trial #19
## 불필요 feature 삭제
## ssscale / pca(n=3) / adasyn

In [110]:
df_19 = df.copy()
df_19 = df_19.drop(columns=['sharing_type', 'has_previous_accident','b2b','accident_ratio','socarpass', 'car_part2'])
df_19

y_target = df_19['fraud_YN']
X_features = df_19.drop('fraud_YN', axis=1)

y_target = df_19['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [111]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=3)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2','component_3']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.tail()


,component_1,component_2,component_3,target
12874,-0.997244,-0.390192,-0.121477,0
12875,-1.084382,1.189193,-1.022075,0
12876,-1.041071,-0.187713,1.265178,0
12877,-1.248356,0.111310,0.408654,0
12878,-1.168033,-1.645701,1.511277,0


In [112]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 

,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.5
DecisionTree,0.997283,0.0,0.0,0.0,0.5
RandomFore,0.997283,0.0,0.0,0.0,0.5
LGBM,0.997283,0.0,0.0,0.0,0.5
SVC,0.997283,0.0,0.0,0.0,0.5


In [113]:
# 복합 샘플링 적용 

from imblearn.over_sampling import ADASYN

ads = ADASYN(random_state=13)
X_train_ads, y_train_ads = ads.fit_sample(X_train, y_train)


model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train_ads, y_train_ads, X_valid, y_valid)

results

,Accuracy,precision,recall,f1,roc
LogiReg,0.725543,0.005650,0.571429,0.011189,0.648696
DecisionTree,0.395575,0.002567,0.571429,0.005112,0.483262
RandomFore,0.943711,0.000000,0.000000,0.000000,0.473141
LGBM,0.943711,0.000000,0.000000,0.000000,0.473141
SVC,0.727873,0.005698,0.571429,0.011283,0.649864


## Trial #20
## 불필요 feature 삭제
## ssscale / pca(n=4) / adasyn

In [114]:
df_20 = df.copy()
df_20 = df_20.drop(columns=['sharing_type', 'has_previous_accident','b2b','accident_ratio','socarpass', 'car_part2'])
df_20

y_target = df_20['fraud_YN']
X_features = df_20.drop('fraud_YN', axis=1)

y_target = df_20['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [116]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=4)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2','component_3',
'component_4']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.tail()


,component_1,component_2,component_3,component_4,target
12874,-0.997216,-0.390764,-0.114709,0.567688,0
12875,-1.084382,1.188923,-1.023409,-1.206782,0
12876,-1.041182,-0.185988,1.264074,0.498973,0
12877,-1.248361,0.111232,0.417679,-0.698917,0
12878,-1.167989,-1.645902,1.503735,0.216451,0


In [117]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 

,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.5
DecisionTree,0.997283,0.0,0.0,0.0,0.5
RandomFore,0.997283,0.0,0.0,0.0,0.5
LGBM,0.997283,0.0,0.0,0.0,0.5
SVC,0.997283,0.0,0.0,0.0,0.5


In [118]:
# 복합 샘플링 적용 

from imblearn.over_sampling import ADASYN

ads = ADASYN(random_state=13)
X_train_ads, y_train_ads = ads.fit_sample(X_train, y_train)


model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train_ads, y_train_ads, X_valid, y_valid)

results

,Accuracy,precision,recall,f1,roc
LogiReg,0.716227,0.006812,0.714286,0.013495,0.715259
DecisionTree,0.701087,0.001307,0.142857,0.002591,0.422733
RandomFore,0.970109,0.000000,0.000000,0.000000,0.486376
LGBM,0.968556,0.000000,0.000000,0.000000,0.485598
SVC,0.719332,0.006887,0.714286,0.013643,0.716816


## Trial #21
## 불필요 feature 삭제
## ssscale / pca(n=5) / adasyn

In [120]:
df_21 = df.copy()
df_21 = df_21.drop(columns=['sharing_type', 'has_previous_accident','b2b','accident_ratio','socarpass', 'car_part2'])
df_21

y_target = df_21['fraud_YN']
X_features = df_21.drop('fraud_YN', axis=1)

y_target = df_21['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [121]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=5)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2','component_3',
'component_4','component_5']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.tail()


,component_1,component_2,component_3,component_4,component_5,target
12874,-0.997281,-0.389816,-0.116335,0.552664,0.174946,0
12875,-1.084352,1.188921,-1.023931,-1.200317,-0.223165,0
12876,-1.041084,-0.187459,1.266949,0.511587,0.279707,0
12877,-1.248457,0.112030,0.416054,-0.701191,1.388933,0
12878,-1.167983,-1.646125,1.503848,0.213860,-1.619860,0


In [122]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 

,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.995730,0.0,0.0,0.0,0.499221
RandomFore,0.997283,0.0,0.0,0.0,0.500000
LGBM,0.997283,0.0,0.0,0.0,0.500000
SVC,0.997283,0.0,0.0,0.0,0.500000


In [123]:
# 복합 샘플링 적용 

from imblearn.over_sampling import ADASYN

ads = ADASYN(random_state=13)
X_train_ads, y_train_ads = ads.fit_sample(X_train, y_train)


model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train_ads, y_train_ads, X_valid, y_valid)

results

,Accuracy,precision,recall,f1,roc
LogiReg,0.678183,0.006010,0.714286,0.011919,0.696185
DecisionTree,0.716227,0.002747,0.285714,0.005442,0.501557
RandomFore,0.977484,0.000000,0.000000,0.000000,0.490074
LGBM,0.974767,0.000000,0.000000,0.000000,0.488712
SVC,0.684394,0.006127,0.714286,0.012151,0.699299
